# Data Visualization Utilities

Because I should probably start standardizing my data exploration.

In [ ]:
import numpy as np
import sys
import os
import pandas as pd
from nltk import word_tokenize
from collections import Counter
from itertools import chain
import matplotlib.pyplot as plt
%matplotlib inline
import re

sys.path.append('..')
from imp import reload
from data import reddit_preprocessor, DataHelper
from data.reddit_preprocessor import *
import json
from pprint import pprint
from jupyterthemes import jtplot

jtplot.style('onedork', ticks=True, fscale=1.5)
jtplot.figsize(x=11., y=8.)
DATA_ROOT = '/home/brandon/Datasets/test_data'
FROM = os.path.join(DATA_ROOT, 'train_from.txt')
TO = os.path.join(DATA_ROOT, 'train_to.txt')

In [ ]:
COL_NAMES = ['inp_sentence', 'resp_sentence']

def make_dataframe(data_dir):
    """
    data_dir: contains train_from.txt, train_to.txt
    """
    from_lines = []
    to_lines = []
    with open(os.path.join(data_dir, 'train_from.txt'), 'r') as from_file:
        with open(os.path.join(data_dir, 'train_to.txt'), 'r') as to_file:
            from_line = from_file.readline()
            to_line = to_file.readline()
            while from_line and to_line:
                from_lines.append(from_line.strip())
                to_lines.append(to_line.strip())
                from_line = from_file.readline()
                to_line = to_file.readline()
            df = pd.DataFrame(np.stack((from_lines, to_lines), 1),
                              columns=COL_NAMES)        
        return df
    
def word_tokenize(df):
    word_freq = {}
    
    # I know. I KNOW.
    sentences = np.squeeze(list(((map(
        DataHelper.word_tokenizer, 
        list(np.expand_dims(df[COL_NAMES[0]].values, 1)))))))
    
    word_freq['from'] = Counter(chain.from_iterable(sentences))
    # Stop judging me.
    sentences = np.squeeze(list(((map(
        DataHelper.word_tokenizer, 
        list(np.expand_dims(df[COL_NAMES[1]].values, 1)))))))
    word_freq['to'] = Counter(chain.from_iterable(sentences))
    
    return word_freq

def plot_freq_dist(word_freq, n):
    words_dict = {}
    for dist in word_freq:
        most_comm = word_freq[dist].most_common(n)
        words, counts = zip(*most_comm)
        words_dict[dist] = words
        counts_series = pd.Series.from_array(counts)

        plt.figure(figsize=(8, 5))
        ax = counts_series.plot(kind='bar')

        ax.set_title('Frequency Distribution: ' + dist)
        ax.set_ylabel('Counts')
        ax.set_xlabel('Words')
        ax.set_xticklabels(words_dict[dist])

    from_words = set(words_dict['from'])
    to_words = set(words_dict['to'])
    common_words = from_words.intersection(to_words)
    common_word_freqs = [
        [word_freq['from'][w] for w in common_words],
        [word_freq['to'][w] for w in common_words]]
    
    ind = np.arange(len(common_words))
    plt.figure(figsize=(8, 5))
    p1 = plt.bar(ind, common_word_freqs[0], width=0.5, color='b')
    p2 = plt.bar(ind, common_word_freqs[1], width=0.5, color='r')
    plt.xticks(ind, common_words)
    plt.legend((p1[0], p2[0]), ('From', 'To'))
    return common_words

In [ ]:
pd.set_option('display.max_colwidth', 10000)
df = make_dataframe(DATA_ROOT)

In [ ]:
pd.set_option('display.max_colwidth', 10000)
df.head(len(df.index))

In [ ]:
word_freq = word_tokenize(df)

In [ ]:
common_words = plot_freq_dist(word_freq, 5)

In [ ]:
common_words